In [68]:
#병렬 크롤링
crawl_range = range(0,45998)
# crawl_range = range(45998,91998)
# crawl_range = range(91998,137994)
# crawl_range = range(137994,183992)
# crawl_range = range(183992,229990)

# crawl_range = range(229990,275988)
# crawl_range = range(275988,321986)
# crawl_range = range(321986,367984)
# crawl_range = range(367984,413982)
# crawl_range = range(413982,459989)

In [65]:
for i in range(0, 11):
    print((459988//10) * i)

0
45998
91996
137994
183992
229990
275988
321986
367984
413982
459980


In [69]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import ijson
import dataframe as df
import numpy as np
import time

#options = Options()
#options.binary_location = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
chromedriver='C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe'
#chromedriver = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
driver = webdriver.Chrome(chromedriver)
time.sleep(2)

In [71]:
# 파일 읽기
f = open(f"./data/data.json", encoding="UTF-8")
objects = ijson.items(f, 'item')
df = pd.DataFrame(objects)

# df = pd.read_csv('./data/성북구_음식점.csv', sep=',', encoding='CP949')

# df = df[['업소명', '소재지도로명', '업태명', '주된음식', '행정동명', '소재지전화번호']]
# df.columns = ['name', 'address', 'cate1', 'cate2', 'dong', 'phone']
# df = df.drop_duplicates(['name'], keep='first')
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.
# df['cate_mix'] = df['cate1'] + df['cate2']

# print(df)

df['naver_keyword'] = df['area'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
# df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다
# for i, keyword in enumerate(df['naver_keyword'].tolist()):
#     print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
for i in crawl_range:
    keyword = df['naver_keyword'][i]
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(0.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(0.5)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(0.5)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

MemoryError: 

In [6]:
from tqdm.notebook import tqdm

naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

# for i, url in enumerate(tqdm(df['naver_map_url'])):
for i in tqdm(crawl_range):
    url = df['naver_map_url'][i]
    driver.get(url)
    sub_driver.get(url+"/review/ugc")
#     time.sleep(0.5)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element_by_css_selector("#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element_by_css_selector("#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements_by_class_name("_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element_by_tag_name('div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

14행 문제가 발생
31행 문제가 발생
37행 문제가 발생
47행 문제가 발생
49행 문제가 발생
65행 문제가 발생
73행 문제가 발생



In [5]:
# 크롤링 파일 저장

import os
from datetime import datetime

today = str(datetime.date(datetime.today()))
file = today + '_test.csv'

if os.path.isfile(file):
    os.remove(file)
df.to_csv(file, encoding='utf-8')
# df.to_csv(today + '_test.csv', encoding='cp949')
print(file)

2021-09-16_test.csv


In [67]:
df

,id,name,branch,area,tel,address,latitude,longitude,category_list,menu_list,bhour_list,review_cnt,review_list,naver_keyword,naver_map_url
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,"[{'category': '아구찜'}, {'category': '포장마차'}]",[],[],0,[],홍대%20Agal,NaN
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,[{'category': '카페'}],[],[],0,[],광주%20Assisy,NaN
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,"[{'category': '피쉬앤칩스'}, {'category': '펍'}]","[{'menu': '메인 (피쉬앤칩스, 오리 가슴살 등)', 'price': 140...","[{'type': 1, 'week_type': 1, 'mon': 0, 'tue': ...",0,[],이태원%20Battered Sole,35162395
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,"[{'category': '레스토랑'}, {'category': '카프레제'}]",[],[],0,[],달맞이길%20Chakyoung,1128117709
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,"[{'category': '디저트카페'}, {'category': '디저트'}]",[],[],0,[],발산역%20Delabobo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459983,459996,행복부대찌개,None,녹양동,031-877-2410,경기도 의정부시 녹양동 154-6,37.757939,127.032758,[],[],[],0,[],녹양동%20행복부대찌개,
459984,459997,행복부페,None,안중읍,031-682-1733,경기도 평택시 안중읍 용성리 718-3,37.002743,126.944991,[],[],[],0,[],안중읍%20행복부페,
459985,459998,행복분식,None,퇴촌면,031-8071-2668,경기도 광주 퇴촌면 도수리 611-6,37.471677,127.315744,[],[],[],0,[],퇴촌면%20행복분식,
459986,459999,행복분식,None,소주동,055-312-3713,경상남도 양산시 소주동 147-1,35.407860,129.158564,[],[],[],0,[],소주동%20행복분식,


In [48]:

for i=5, keyword in enumerate(df['naver_keyword'].tolist()):
    if i > 10:
        break
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)

SyntaxError: invalid syntax (<ipython-input-48-9de3b24ec52f>, line 1)

In [54]:
ran = range(10, 15)
# split_keyword = df['naver_keyword']
for i in ran:
    keyword =df['naver_keyword'][i]
    print(keyword)

월곶%20국수이야기
역삼역%20깐부치킨
수영역%20모이바바
유성%20미랑
전포동%20써리힐


In [47]:
df

pandas.core.series.Series

In [ ]:
# %%cmd
# head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json
# 안되니까 git bash로 확인
# 한줄 데이터라 byte 단위로 json 구조 확인
# head $ head data.json -c 1000

In [ ]:
# #데이터가 너무 커서 블록단위로 메모리에 올리는 ijson 사용
# import ijson
# import pandas

# with open('C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json','r',encoding='utf-8') as f:
#     obj = ijson.items(f, 'columns.item')
#     columns = list(obj)
#     print(columns[0])